In [86]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout, BatchNormalization
import numpy as np
import matplotlib.pyplot as plt
import wandb
from wandb.keras import WandbCallback
import os

In [ ]:
if os.path.exists('best_model.h5'):
    os.remove('best_model.h5')

In [2]:
seed = 100
tf.random.set_seed(seed)
np.random.seed(seed)

In [3]:
#wandb.login()

### Downloading and unzipping iNaturalist Dataset

In [4]:
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip

In [5]:
!unzip "./nature_12K.zip"

### Preparing data for training

In [6]:
def train_dataset(augmentation=False, batch_size=64):
    dir_train = './inaturalist_12K/train'
    dir_test = './inaturalist_12K/val'

    if augmentation:
        train_datagen = ImageDataGenerator(rescale=1./255,
                                          zoom_range=0.3,
                                          rotation_range=50,
                                          brightness_range=(0.2, 0.8),
                                          shear_range=0.2,
                                          width_shift_range=0.1,
                                          height_shift_range=0.2,
                                          horizontal_flip=True,
                                          vertical_flip=True,
                                          validation_split=0.1,)
        test_datagen = ImageDataGenerator(rescale=1./255)

    else:
        train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)
        test_datagen = ImageDataGenerator(rescale=1./255)

    train = train_datagen.flow_from_directory(dir_train, target_size=(200, 200), batch_size=batch_size, subset="training")
    val = train_datagen.flow_from_directory(dir_train, target_size=(200, 200), batch_size=batch_size, subset="validation")
    test = test_datagen.flow_from_directory(dir_test, target_size=(200, 200), batch_size=batch_size)
    
    return train, val, test;

In [7]:
def CNN(n_filters, filter_multiplier, dropout, batch_norm, dense_size, act_func= "relu", n_classes=10, image_size=200):
    
    model = Sequential()
    for i in range(5):
        filter_dim = 10 - (2*i)
        filter_size = (filter_dim, filter_dim)
        if i==0:
            model.add(Conv2D(n_filters, filter_size, input_shape=(image_size, image_size, 3), data_format="channels_last"))
        else:
            model.add(Conv2D(n_filters, filter_size))
        if batch_norm:
            model.add(BatchNormalization())
        if act_func == "relu":
            model.add(Activation(act_func))
        if act_func == "leaky":
            model.add(keras.layers.LeakyReLU(alpha=0.3))
        model.add(MaxPooling2D(pool_size=(2,2)))
        num_filters = int(n_filters * filter_multiplier)
    
    model.add(Flatten())
    model.add(Dense(dense_size))
    model.add(Dropout(dropout))
    if act_func == "relu":
        model.add(Activation("relu"))
    if act_func == "leaky":
        model.add(keras.layers.LeakyReLU(alpha=0.3))
    model.add(Dense(n_classes))
    model.add(Activation("softmax"))

    return model

In [8]:
model.summary()

### Visualizing some training images

In [10]:
train, val, test = train_dataset(augmentation=False, batch_size=64)
img = train.next()

In [11]:
idx_to_class = {0: 'Amphibia', 1: 'Animalia', 2: 'Arachnida', 3: 'Aves', 4: 'Fungi', 
                  5: 'Insecta', 6: 'Mammalia', 7: 'Mollusca', 8: 'Plantae', 9: 'Reptilia'}

plt.figure(figsize=(15,15))
for i in range(64):
    plt.subplot(8,8,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(img[0][i])
    plt.xlabel(idx_to_class[np.argmax(img[1][i])])
plt.show()

### Function to set WandB run name

In [12]:
def setWandbName(n_filters, filter_multiplier, augment, dropout, batch_norm):
    
    batch_norm_dict = {True: "Y", False: "N"}
    augment_dict = {True: "Y", False: "N"}

    name = "_".join(["num", str(n_filters), "org", str(filter_multiplier), "aug", augment_dict[augment],
                      "drop", str(dropout), "norm", batch_norm_dict[batch_norm]])
    
    return name;

### Function to train the dataset

In [14]:
def train_wandb(config= None):

    '''best_config = {
        "n_filters": 64,
        "filter_multiplier": 2,
        "augment_data": True,
        "dropout": 0.3,
        "batch_norm": True,
        "epochs": 20,
        "dense_size": 32,
        "lr": 0.001
    }'''
    
    wandb.init(project="Convolutional Neural Networks", entity="cs21s048-cs21s058")
    config = wandb.config
    print(config.augment_data)
    wandb.run.name = setWandbName(config.n_filters, config.filter_multiplier, config.augment_data, config.dropout, config.batch_norm)

    train, val, test = train_dataset(augmentation=config.augment_data, batch_size=config.batch_size)

    model = CNN(n_filters=config.n_filters, filter_multiplier=config.filter_multiplier,
                      dropout= config.dropout, batch_norm = config.batch_norm, dense_size= config.dense_size)
    model.compile(optimizer=keras.optimizers.Adam(config.lr), loss="categorical_crossentropy", metrics="categorical_accuracy")
    model.fit(train, epochs=config.epochs, validation_data=val, callbacks=[WandbCallback()])

In [15]:
def train(config= None):

    '''best_config = {
        "n_filters": 16,
        "filter_multiplier": 2,
        "augment_data": True,
        "dropout": 0.3,
        "batch_norm": True,
        "batch_size" : 64,
        "epochs": 10,
        "dense_size": 128,
        "lr": 0.001
        "activation" : "leaky"
    }'''

    train, val, test = train_dataset(augmentation=True, batch_size=64)

    model = CNN(n_filters=32, filter_multiplier=2,
                      dropout= 0.5, batch_norm = True, dense_size= 128,act_func= "relu")
    model.compile(optimizer=keras.optimizers.Adam(0.001), loss="categorical_crossentropy", metrics="categorical_accuracy")
    model.fit(train, epochs=10, validation_data=val)
    print("Testing Model: ")
    model.evaluate(test, batch_size=64)
    
    model.save("best_model.h5")
    model = keras.models.load_model("best_model.h5")

### Predict Function

In [16]:
def predict(model, test_data):
    
    predictions = model(test[0][0])
    model.evaluate(test_data, batch_size=64)
    #plot_images_predicted(test_data, predictions, test_data[0][1])
    return predictions
    #sample_num = 11
    #plot_filters(model, test_generator, sample_num)

### Setting up wandb sweep

In [17]:
sweep_config = {
    "name": "Final Sweep(Bayesian)",
    "description": "Tuning hyperparameters",
    'metric': {
      'name': 'val_categorical_accuracy',
      'goal': 'maximize'
  },
    "method": "bayes",
    "project": "CS6910_Assignment2",
    "parameters": {
        "n_filters": {
        "values": [16, 32, 64]
        },
        "filter_multiplier": {
            "values": [0.5, 1, 2]
        },
        "augment_data": {
            "values": [True]
        },
        "dropout": {
            "values": [0.3, 0.5]
        },
        "batch_norm": {
            "values": [False, True]
        },
        "epochs": {
            "values": [5, 10]
        },
        "dense_size": {
            "values": [32, 64, 128]
        },
        "lr": {
            "values": [0.01, 0.001]
        },
        "batch_size": {
            "values": [64, 128, 256]
        },
        "activation": {
            "values": ["relu", "leaky"]
        },
    }
}

# creating the sweep
#sweep_id = wandb.sweep(sweep_config, project="Convolutional Neural Networks", entity="cs21s048-cs21s058")

In [ ]:
#wandb.agent("5qv1kqvj", function=train_wandb, count=50)

In [18]:
train()

In [19]:
model = keras.models.load_model("best_model.h5")

In [ ]:
predictions = predict(model,test)

### Plotting test data images with predicted labels

In [ ]:
labels= test[0][1]
plt.figure(figsize=(15,15))
for i in range(30):
        img = test[0][0][i]
        plt.subplot(10,3,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.tight_layout()
        plt.imshow(img)
        #plt.subplots_adjust(bottom=0.2, 
        #            top=0.9, 
        #            wspace=0.4, 
        #            hspace=0.4)
        plt.xlabel("Predicted: " + idx_to_class[np.argmax(predictions, axis=1)[i]] + "\nLabel: " + idx_to_class[np.argmax(labels, axis=1)[i]])
plt.savefig('q4b.png')
plt.show()

### Visualize filters of first layer

In [37]:
#Print sizes of all convolution layers:

for layer in model.layers:
    if 'conv' in layer.name:
        filters , bias = layer.get_weights()
        print(layer.name , filters.shape)

### Visualising filters from 1st layer

In [95]:
# get weights from the second hidden layer
filters , bias = model.layers[0].get_weights()

# normalize filter values to 0-1 so we can visualizing
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)

n_filters =32
ix=1
fig = plt.figure(figsize=(20,20))

fig, ax = plt.subplots(8, 4, figsize=(15,20))
for i in range(filters.shape[-1]):
        ax[int(i/4), i%4].imshow(filters[:, :, :, i])
        ax[int(i/4), i%4].axis('off')
        
plt.savefig('q4c.png')

### Visualising feature maps

In [97]:
feature_map_model = Model(inputs=model.inputs, outputs=model.layers[0].output)

#taking an image from test data
plt.imshow(test[0][0][7])
plt.axis('off')

feature_maps = feature_map_model(test[0][0])
fig, ax = plt.subplots(4, 8, figsize=(12,6))
for i in range(feature_maps.shape[-1]):
    ax[int(i/8), i%8].imshow(feature_maps[7, :, :, i], cmap='gray')
    ax[int(i/8), i%8].axis('off')

plt.savefig('q4c_2.png')

### Guided Backpropagation

In [104]:
def guided_backprop(model_path, n_images, test):

    # loading the model
    model = tf.keras.models.load_model(model_path)

    # take upto 5th convolutional layer
    model_upto_conv5 = tf.keras.models.Model([model.inputs],[model.get_layer("conv2d_4").output])

    # creating a custom gradient to track actual non-zero gradients
    @tf.custom_gradient
    def guidedRelU(x):
        def grad(dy):
            return tf.cast(dy>0, tf.float32) * tf.cast(x>0, tf.float32) * dy
        return tf.nn.relu(x), grad

    for layer in model.layers[1:]:
        if hasattr(layer, 'activation') and layer.activation==tf.keras.activations.relu:
            layer.activation = guidedRelU

    # plotting the images
    fig, axs = plt.subplots(n_images, 2, figsize=(2*4, n_images*4))
    inputs = tf.convert_to_tensor(test[0][0][:n_images], dtype=tf.float32)
    
    for i in range(n_images):

        with tf.GradientTape() as tape:
            input = tf.expand_dims(inputs[i], 0)
            tape.watch(input)
            output = model_upto_conv5(input)[0]
        
        gradients = tape.gradient(output,input)[0]

        axs[i][0].set_title("Original Image")
        axs[i][0].imshow(inputs[i])
        axs[i][1].set_title("Guided backpropagation")
        axs[i][1].imshow(gradients)

    plt.show()

In [105]:
guided_backprop('best_model.h5', 10, test)